In [7]:
using BenchmarkTools: @ballocated
using LinearAlgebra: I, norm, triu, tril, tr, diagm, diag, qr, eigvals, istriu, dot
using CairoMakie

In [78]:
function arnoldi(A, q1, kmax)
    H = zeros(kmax, kmax)
    Q = zeros(size(q1, 1), kmax)

    Q[:, 1] = q1 / norm(q1)
    for k = 1:kmax
        if k > 1
            Q[:, k] = q1/ H[k, k-1]
        end

        q1 = A * Q[:, k]
        for i = 1:k
            H[i, k] = 0
            for j = 1:size(q1, 1)
                H[i, k] += Q[j, i] * q1[j]
            end
            q1 -= H[i, k] * Q[:, i]
        end

        if k < kmax
            H[k+1, k] = norm(q1)
        end
    end
    return H
end

m = 5
A = rand(m,m)
A = A * A' 
display(A)
H = arnoldi(A, randn(m), m)
display(H)

display(istriu(H[2 : end, 1 : (end - 1)]))
display(eigvals(H) ≈ eigvals(A))

5×5 Matrix{Float64}:
 1.68495   0.928051  1.10032  0.622746  0.986485
 0.928051  0.640303  1.03377  0.61696   0.677422
 1.10032   1.03377   2.27118  1.51781   1.1757
 0.622746  0.61696   1.51781  1.17884   0.741031
 0.986485  0.677422  1.1757   0.741031  0.798362

5×5 Matrix{Float64}:
 0.819078  1.81353   7.91034e-16  -1.77946e-14  3.29511e-14
 1.81353   4.48795   0.677473     -4.3604e-14   8.05948e-14
 0.0       0.677473  1.11983       0.0739697    1.03878e-14
 0.0       0.0       0.0739697     0.040896     0.0199846
 0.0       0.0       0.0           0.0199846    0.105883

true

true

In [80]:
function lanczos(A, q1, kmax)
    T = zeros(kmax, kmax)
    r = copy(q1[:, 1])
    beta = norm(r)
    q1 = zeros(size(A, 1))
    
    for k = 1:kmax
        q0 = copy(q1)
        q1 = r/beta
        r = A * q1
        alpha = 0
        for j = 1:size(r, 1)
            alpha += q1[j] * r[j]
        end
        T[k, k] = alpha
        if k > 1
            T[k-1, k] = beta
            T[k, k-1] = beta
        end
        r = r - alpha*q1 - beta * q0
        beta = norm(r)
    end
    return diag(T), diag(T,1)
end

m = 5
A = rand(m,m)
A = A * A' 
a, b = lanczos(A, randn(m), m)

H = diagm(-1 => b, 0 => a, 1 => b)

eigvals(H) ≈ eigvals(A) 

true

In [116]:
m = 8
A = rand(m,m)
Q,_ = qr(A)
λ = 3 .^ range(0,m-1)
Σ = diagm(λ)
T = Q'Σ*Q

max_eigs_A = zeros(m)
max_eigs_L = zeros(m)

# display(T)
for kmax = 1:m
    H_A = arnoldi(T, randn(m), kmax)
    max_eigs_A[kmax] = sort(eigvals(H_A), rev=true)[1]

    a, b = lanczos(T, randn(m), kmax)
    H_L = diagm(-1 => b, 0 => a, 1 => b)
    max_eigs_L[kmax] = sort(eigvals(H_L), rev=true)[1]
end

max_eig_A_error = abs.(max_eigs_A .- 3^(m-1))/3^(m-1)
max_eig_L_error = abs.(max_eigs_L .- 3^(m-1))/3^(m-1)

error_plot = scatter(1:m, max_eig_A_error, label="Arnoldi", axis=(yscale=log10, xlabel="iteration, k", ylabel="Normalized Eigenvalue Error"))
scatter!(1:m, max_eig_L_error, label="Lanczos")
axislegend(position=:rt)
save("compare.pdf", error_plot)

CairoMakie.Screen{PDF}


In [190]:
m = 8
A = rand(m,m)
Q,_ = qr(A)
λ = [1, 2, 3, 4, 5, 6, 7, 1e5]
Σ = diagm(λ)
T = Q'Σ*Q

# max_eigs_A = zeros(m)
# max_eigs_L = zeros(m)

# # display(T)
# for kmax = 1:m
#     H_A = arnoldi(T, randn(m), kmax)
#     max_eigs_A[kmax] = sort(eigvals(H_A), rev=true)[1]

#     a, b = lanczos(T, randn(m), kmax)
#     H_L = diagm(-1 => b, 0 => a, 1 => b)
#     max_eigs_L[kmax] = sort(eigvals(H_L), rev=true)[1]
# end

# max_eig_A_error = abs.(max_eigs_A .- 3^(m-1))/3^(m-1)
# max_eig_L_error = abs.(max_eigs_L .- 3^(m-1))/3^(m-1)

# error_plot = scatter(1:m, max_eig_A_error, label="Arnoldi", axis=(yscale=log10, xlabel="iteration, k", ylabel="Normalized Eigenvalue Error"))
# scatter!(1:m, max_eig_L_error, label="Lanczos")
# axislegend(position=:rt)
# save("compare.pdf", error_plot)
# display(λ)
a, b = lanczos(T, randn(m), m)
H_L = diagm(-1 => b, 0 => a, 1 => b)
display(eigvals(H_L))
display(eigvals(T))


8-element Vector{Float64}:
      1.0059152000327536
      2.021443685050605
      3.4066062087189697
      4.36702436636957
      5.985918579747498
      6.99621700967154
 100000.00000000001
 100000.00000000007

8-element Vector{Float64}:
      0.9999999999991087
      1.9999999999986766
      3.0000000000024776
      4.000000000000933
      5.000000000001729
      6.000000000000285
      6.999999999997204
 100000.00000000003